In [1]:
# type:ignore
import time
import sys
import io
import pandas as pd

from gurobipy import *
from functions import *
from openpyxl import load_workbook

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# parâmetros
instances = [
             'instancia1',
             'instancia2',
             'instancia3',
             'instancia4',
             'instancia5',
             'instancia6',
             'instancia7',
             'instancia8',
             'instancia9',
             'instancia10' 
            ]

Qs = [561,637,563,561,561,565,491,493,563,493]

In [ ]:
path_excel = 'C:/Wilmer/tesis/Instancias/resumo.xlsx'
wb = load_workbook(path_excel)
hoja = wb.active

for instance in instances:
    
    path = 'C:/Wilmer/tesis/Instancias/'+instance
    Q = Qs[instances.index(instance)]

    modelos = [
               BaseModel,
               BaseModel_Fulfillments,
               BaseModel_Skiplagging,
               BaseModel_Fulfillments_Skiplagging,

               HierarBehavioralModel,
               HierarBehavioralModel_Fulfillments,
               HierarBehavioralModel_Skiplagging,
               HierarBehavioralModel_Fulfillments_Skiplagging,

               PercentBehavioralModel,
               PercentBehavioralModel_Fulfillments,
               PercentBehavioralModel_Skiplagging,
               PercentBehavioralModel_Fulfillments_Skiplagging
            ]

    for classeModelo in modelos:

        # criar modelo
        start_time_cria_model = time.time()

        modelo = classeModelo(
            path_dem = path + '/demanda.csv',
            path_preco = path + '/preco.csv',
            path_rota1 = path + '/rota.csv',
            Q = Q,
            perio = 0
        )

        model, A, X, Y, BY, BX, BL, P, d, perio, indexCombiDem = modelo.create_model()

        end_time_cria_model = time.time()


        # Redigir a saída padrão para um arquivo
        old_stdout = sys.stdout
        new_stdout = io.StringIO()
        sys.stdout = new_stdout


        # Optimizar o modelo
        start_time_opt = time.time()
        model.optimize()
        end_time_opt = time.time()

        # Calcular tempos
        tempoCriacao = end_time_cria_model - start_time_cria_model
        tempoOpt = end_time_opt - start_time_opt

        if model.status == GRB.OPTIMAL:
            # Salvar solução
            a = save_solution(model, BX, BL, P, d, X, Y, A, BY, perio, classeModelo.__name__, indexCombiDem, path)

            print("Modelo: ", classeModelo.__name__)
            print("Instancia: ", instance)
            print("FO: ", model.ObjVal)
            print("FO Relaxada: ", model.ObjBound)
            print("Tempo Criação: ", tempoCriacao)
            print("Tempo da optimização: ", tempoOpt)
        else:
            print("Modelo: ", classeModelo.__name__)
            print("Instancia: ", instance)
            print("FO: ", "Infac")
            print("FO Relaxada: ", model.ObjBound)
            print("Tempo Criação: ", tempoCriacao)
            print("Tempo da optimização: ", tempoOpt)

        # Voltar a saída padrão
        sys.stdout = old_stdout

        # Obter a saída da otimização
        output = new_stdout.getvalue()

        # Salvar a saída num arquivo de texto
        with open( path + '/saidaExecucao'+ classeModelo.__name__ +'.txt', 'w') as f:
            f.write(output)

        # Salvar em excel um resumo dos resultados
        if model.status == GRB.OPTIMAL:
            hoja.append([instance, classeModelo.__name__, model.NumVars, tempoCriacao, tempoOpt, model.NodeCount, model.IterCount, model.SolCount, model.MIPGap, model.ObjBound, model.ObjVal])
        else:
            hoja.append([instance, classeModelo.__name__, model.NumVars, tempoCriacao, tempoOpt, "-", "-", "-", "-", "-",'Infac'])
        wb.save(path_excel)

        print("Termino a Instancia: " + instance + ", usando o modelo: " + classeModelo.__name__)

In [4]:
# attrs = ['Authorizations','Assignments','Demanda','Preco']

# graf2 = grafica(a,attrs, 4, 15, 4, 2)
# graf1 = grafica(a,attrs, 4, 15, 4, 1)
# graf0 = grafica(a,attrs, 4, 15, 4, 0)

# graf2.savefig(path+'/img2.png', dpi=300, transparent=True)
# graf1.savefig(path+'/img1.png', dpi=300, transparent=True)
# graf0.savefig(path+'/img0.png', dpi=300, transparent=True)
